<a href="https://colab.research.google.com/github/denisich2001/DataScienceLabs/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_2_%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Лабораторная работа 2**
**Кирьянов Денис группа N3248** 

**Задание 1**

Загружаем файл

In [ ]:
import pandas as pd
data = pd.read_csv("songs.csv")

In [ ]:
data.head()

,genre,lyrics,SongInfo
0,Christian,"Who am I, that the Lord of all the earth Woul...",CASTING CROWNS - WHO AM I LYRICS
1,Christian,Glory Revealed By His Wounds He was pierced ...,GLORY REVEALED - BY HIS WOUNDS LYRICS
2,Christian,Lord of heaven and earth Lord of all creation...,CAEDMON'S CALL - GOD OF WONDERS LYRICS
3,Christian,I can only imagine what it will be like When ...,MERCYME - I CAN ONLY IMAGINE LYRICS
4,Christian,I am not skilled to understand What God has w...,AARON SHUST - MY SAVIOR MY GOD LYRICS


Оставляем только нужные для обработки колонки

In [ ]:
columns = data[['genre','lyrics']]
columns = columns[(columns.genre == 'R&B')|(columns.genre == 'Christian')]
columns

,genre,lyrics
0,Christian,"Who am I, that the Lord of all the earth Woul..."
1,Christian,Glory Revealed By His Wounds He was pierced ...
2,Christian,Lord of heaven and earth Lord of all creation...
3,Christian,I can only imagine what it will be like When ...
4,Christian,I am not skilled to understand What God has w...
...,...,...
553,R&B,"Ha I dont care ha, about your past I just wan..."
554,R&B,Hoverin by my suitcase Tryin to find a warm ...
555,R&B,I dont know why I love you like I do After a...
556,R&B,"C. C. Rider Elvis Presley Well now see., C. ..."


Создаем функцию для обработки текстовых данных.
На нее мы будем ссылаться в будущем

In [ ]:
def preproc(columns):
  #Перевод в нижний регистр
  lowered = columns['lyrics'].str.lower()
  columns['lowered'] = lowered


  #Токенизация(разбиваем на отдельные слова)
  import nltk
  from nltk.tokenize import word_tokenize
  nltk.download('punkt')

  tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
  columns['tokened'] = tokened


  #Удаляем стоп-слова
  nltk.download('stopwords')
  from nltk.corpus import stopwords

  withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in stopwords.words('english')])
  without_stop = []
  for a in withoutstop:    
      without_stop.append(", ".join(a))
  columns['without_stop'] = without_stop
    

  #Производим лемматизацию
  from nltk.stem import WordNetLemmatizer 
  nltk.download('wordnet')
  lemmatizer = WordNetLemmatizer()
  lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
  lemma = []
  for a in lemmatized:    
      lemma.append(", ".join(a))
  columns['result'] = lemma

  return columns

Обрабатываем наши данные с помощью функции описанной выше

In [ ]:
columns = preproc(columns)
columns

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,genre,lyrics,lowered,tokened,without_stop,result
0,Christian,"Who am I, that the Lord of all the earth Woul...","who am i, that the lord of all the earth woul...","[who, am, i, ,, that, the, lord, of, all, the,...",",, lord, earth, would, care, know, name, would...",",, lord, earth, would, care, know, name, would..."
1,Christian,Glory Revealed By His Wounds He was pierced ...,glory revealed by his wounds he was pierced ...,"[glory, revealed, by, his, wounds, he, was, pi...","glory, revealed, wounds, pierced, transgressio...","glory, revealed, wounds, pierced, transgressio..."
2,Christian,Lord of heaven and earth Lord of all creation...,lord of heaven and earth lord of all creation...,"[lord, of, heaven, and, earth, lord, of, all, ...","lord, heaven, earth, lord, creation, lord, hea...","lord, heaven, earth, lord, creation, lord, hea..."
3,Christian,I can only imagine what it will be like When ...,i can only imagine what it will be like when ...,"[i, can, only, imagine, what, it, will, be, li...","imagine, like, walk, side, imagine, eyes, see,...","imagine, like, walk, side, imagine, eyes, see,..."
4,Christian,I am not skilled to understand What God has w...,i am not skilled to understand what god has w...,"[i, am, not, skilled, to, understand, what, go...","skilled, understand, god, willed, ,, god, plan...","skilled, understand, god, willed, ,, god, plan..."
...,...,...,...,...,...,...
553,R&B,"Ha I dont care ha, about your past I just wan...","ha i dont care ha, about your past i just wan...","[ha, i, dont, care, ha, ,, about, your, past, ...","ha, dont, care, ha, ,, past, want, ho, ,, love...","ha, dont, care, ha, ,, past, want, ho, ,, love..."
554,R&B,Hoverin by my suitcase Tryin to find a warm ...,hoverin by my suitcase tryin to find a warm ...,"[hoverin, by, my, suitcase, tryin, to, find, a...","hoverin, suitcase, tryin, find, warm, place, s...","hoverin, suitcase, tryin, find, warm, place, s..."
555,R&B,I dont know why I love you like I do After a...,i dont know why i love you like i do after a...,"[i, dont, know, why, i, love, you, like, i, do...","dont, know, love, like, changes, put, stole, m...","dont, know, love, like, changes, put, stole, m..."
556,R&B,"C. C. Rider Elvis Presley Well now see., C. ...","c. c. rider elvis presley well now see., c. ...","[c., c., rider, elvis, presley, well, now, see...","c., c., rider, elvis, presley, well, see., ,, ...","c., c., rider, elvis, presley, well, see., ,, ..."


Разбиваем данные на 2 набора: для тренировки и для тестирования 

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(columns.result, columns.genre, train_size = 0.7)

Производим Векторизацию

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

Обучаем и тестируем модель

In [ ]:
#обучаем Байесовский классификатор
from sklearn.naive_bayes import MultinomialNB 
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)

#предсказываем и получаем оценку точности
from sklearn.metrics import * 
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

   Christian       0.93      0.96      0.95        27
         R&B       0.96      0.93      0.95        29

    accuracy                           0.95        56
   macro avg       0.95      0.95      0.95        56
weighted avg       0.95      0.95      0.95        56



**Задание 2**

Добавляем текста песен для тестирования предыдущей модели

In [ ]:
song_1_lyrics = '''When I see you
I run out of words to say
I wouldn't leave you
'Cause you're that type of girl to make me stay

I see the guys tryna holla, girl I don't wanna bother you
'Cause you're independent and you got my attention
Can I be your baby father? Girl, I just wanna show you
That I love what you are doin' hun

I see you in the club, you gettin' down girl
I wanna get with you, yeah
I see you in the club, you showin' thugs love
I wanna get with you

You're so beautiful
So damn beautiful
Said you're so beautiful
So damn beautiful

You're so beautiful, beautiful
Beautiful, beautiful
You're so beautiful, beautiful
Beautiful, beautiful
You're so beautiful

Like the clouds you
Drift me away, far away, yeah
And like the sun you
Brighten the day, you brighten my day, yeah

I never wanna see you cry cry cry
And I never wanna tell a lie lie lie
Said I never wanna see you cry cry cry
And I never wanna tell a lie lie lie

I see you in the club, you gettin' down girl
I wanna get with you, yeah
I see you in the club, you showin' thugs love
I wanna get with you

You're so beautiful
So damn beautiful
Said you're so beautiful
So damn beautiful

You're so beautiful, beautiful
Beautiful, beautiful
You're so beautiful, beautiful
Beautiful, beautiful
You're so beautiful

Kardinal told you, whether the sky blue or yellow
This fella ain't that mellow if it ain't about you!
Hourglass shape make a place go 'woo!'
Waistline makes my soldier salute

I'm a brute, high from your high heel game
High heels push up ya ass last name
And you livin' in the fast lane, eyes like an angel
Goddess, watch my gal and how she undress

Spotless, heartless, bad to the bone
Make me wanna go put me in the triple-X zone
Lames don't know how to talk to you
So let me walk with you, hold my hand

I'ma spend them grands but after you undress
Not like a hooker but more like a Princess
Queen, Empress, President
Pull anywhere you go on Earth 'cause you're beautiful, okay?

I see you in the club, you gettin' down girl
I wanna get with you, yeah
I see you in the club, you showin' thugs love
I wanna get with you

You're so beautiful
So damn beautiful
Said you're so beautiful
So damn beautiful

You're so beautiful, beautiful
Beautiful, beautiful
You're so beautiful, beautiful
Beautiful, beautiful
You're so beautiful

Where'd you come from? You're outta this world to me
You're a symbol of what every beautiful woman should be, ooh wee

I never wanna see you cry cry cry
And I never wanna tell a lie lie lie
Said I never wanna see you cry cry cry
And I never wanna tell a lie lie lie

I see you in the club, you gettin' down girl
I wanna get with you, yeah
I see you in the club, you showin' thugs love
I wanna get with you

You're so beautiful
So damn beautiful
Said you're so beautiful
So damn beautiful
You're so beautiful'''

song_2_lyrics = '''
They say sometimes you win some
Sometimes you lose some
And right now, right now I'm losing bad
I've stood on this stage night after night
Reminding the broken it'll be alright
But right now, oh right now I just can't

It's easy to sing
When there's nothing to bring me down
But what will I say
When I'm held to the flame
Like I am right now

I know You're able and I know You can
Save through the fire with Your mighty hand
But even if You don't
My hope is You alone

They say it only takes a little faith
To move a mountain
Good thing
A little faith is all I have right now
But God, when You choose
To leave mountains unmovable
Give me the strength to be able to sing
It is well with my soul

I know You're able and I know You can
Save through the fire with Your mighty hand
But even if You don't
My hope is You alone
I know the sorrow, and I know the hurt
Would all go away if You'd just say the word
But even if You don't
My hope is You alone

You've been faithful, You've been good
All of my days
Jesus, I will cling to You
Come what may
‘Cause I know You're able
I know You can

I know You're able and I know You can
Save through the fire with Your mighty hand
But even if You don't
My hope is You alone
I know the sorrow, and I know the hurt
Would all go away if You'd just say the word
But even if You don't
My hope is You alone

It is well with my soul
It is well, it is well with my soul
'''
song_1_genre = "R&B"
song_2_genre = "Christian"

Создаем из песен датасет и обрабатываем его с помощью описанной выше фукнции preproc()

In [ ]:
columns2 = pd.DataFrame(data = {"genre":[song_1_genre,song_2_genre],"lyrics":[song1_lyrics,song_2_lyrics]})
columns2 = preproc(columns2)
columns2

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,genre,lyrics,lowered,tokened,without_stop,result
0,R&B,When I see you\nI run out of words to say\nI w...,when i see you\ni run out of words to say\ni w...,"[when, i, see, you, i, run, out, of, words, to...","see, run, words, say, would, n't, leave, 'caus...","see, run, words, say, would, n't, leave, 'caus..."
1,Christian,\nThey say sometimes you win some\nSometimes y...,\nthey say sometimes you win some\nsometimes y...,"[they, say, sometimes, you, win, some, sometim...","say, sometimes, win, sometimes, lose, right, ,...","say, sometimes, win, sometimes, lose, right, ,..."


Тестируем модель на векторизированных данных

In [ ]:
pred2 = clf.predict(vectorizer.transform(columns2['result']))
print(classification_report(columns2['genre'], pred2))

              precision    recall  f1-score   support

   Christian       1.00      1.00      1.00         1
         R&B       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



**Задание 3**

Загружаем данные, создаем из них датасет и обрабатываем текста функцией preproc()

In [ ]:
data2 = pd.read_csv("dataset-lyrics-musics-mini.csv")
columns = data2[['cantorNome', 'letra']]
columns = columns[(columns.cantorNome == "david-bowie")|(columns.cantorNome == "paul-mccartney")]
columns['lyrics'] = columns['letra']
columns = preproc(columns)
columns

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,cantorNome,letra,lyrics,lowered,tokened,without_stop,result
0,david-bowie,"I, I will be king. And you, you will be queen....","I, I will be king. And you, you will be queen....","i, i will be king. and you, you will be queen....","[i, ,, i, will, be, king, ., and, you, ,, you,...",",, king, ., ,, queen, ., though, nothing, driv...",",, king, ., ,, queen, ., though, nothing, driv..."
1,david-bowie,"Didn't know what time it was,. The lights were...","Didn't know what time it was,. The lights were...","didn't know what time it was,. the lights were...","[did, n't, know, what, time, it, was, ,, ., th...","n't, know, time, ,, ., lights, low, ., leaned,...","n't, know, time, ,, ., lights, low, ., leaned,..."
2,david-bowie,Ground control to Major Tom. Ground control to...,Ground control to Major Tom. Ground control to...,ground control to major tom. ground control to...,"[ground, control, to, major, tom, ., ground, c...","ground, control, major, tom, ., ground, contro...","ground, control, major, tom, ., ground, contro..."
3,david-bowie,It's a god-awful small affair. To the girl wit...,It's a god-awful small affair. To the girl wit...,it's a god-awful small affair. to the girl wit...,"[it, 's, a, god-awful, small, affair, ., to, t...","'s, god-awful, small, affair, ., girl, mousy, ...","'s, god-awful, small, affair, ., girl, mousy, ..."
4,david-bowie,I know when to go out. And when to stay in. Ge...,I know when to go out. And when to stay in. Ge...,i know when to go out. and when to stay in. ge...,"[i, know, when, to, go, out, ., and, when, to,...","know, go, ., stay, ., get, things, done, ., ca...","know, go, ., stay, ., get, things, done, ., ca..."
...,...,...,...,...,...,...,...
942,paul-mccartney,He's just a young boy looking for a way to fin...,He's just a young boy looking for a way to fin...,he's just a young boy looking for a way to fin...,"[he, 's, just, a, young, boy, looking, for, a,...","'s, young, boy, looking, way, find, love, ., n...","'s, young, boy, looking, way, find, love, ., n..."
943,paul-mccartney,How can I hope to reach your love. Help me to ...,How can I hope to reach your love. Help me to ...,how can i hope to reach your love. help me to ...,"[how, can, i, hope, to, reach, your, love, ., ...","hope, reach, love, ., help, discover, ., 're, ...","hope, reach, love, ., help, discover, ., 're, ..."
944,paul-mccartney,I like it. Please don't take my heart away. It...,I like it. Please don't take my heart away. It...,i like it. please don't take my heart away. it...,"[i, like, it, ., please, do, n't, take, my, he...","like, ., please, n't, take, heart, away, ., 's...","like, ., please, n't, take, heart, away, ., 's..."
945,paul-mccartney,Yvonne is the one I´ve been counting on. She s...,Yvonne is the one I´ve been counting on. She s...,yvonne is the one i´ve been counting on. she s...,"[yvonne, is, the, one, i´ve, been, counting, o...","yvonne, one, i´ve, counting, ., said, long, yo...","yvonne, one, i´ve, counting, ., said, long, yo..."


Разбиваем данные для тренировки и тестирования и векторизируем текст

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(columns.result, columns.cantorNome, train_size = 0.7)


#Векторизация
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

Тренируем модель и проверяем его точность на тестовых данных

In [ ]:
#обучаем Баесовский классификатор
from sklearn.naive_bayes import MultinomialNB 
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)

#предсказываем и получаем оценку точности
from sklearn.metrics import * 
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(y_test, pred))

                precision    recall  f1-score   support

   david-bowie       0.79      0.73      0.76       154
paul-mccartney       0.71      0.77      0.74       131

      accuracy                           0.75       285
     macro avg       0.75      0.75      0.75       285
  weighted avg       0.75      0.75      0.75       285

